# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. 

In [44]:
#imported libraries:
import numpy as np
import pandas as pd
import nltk

In [45]:
credit_info = pd.read_csv('/datasets/credit_scoring_eng.csv')
print(credit_info.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [46]:
#looking through the content of some columns:
inuque_values=[credit_info['dob_years'].unique(),
credit_info['children'].unique(),credit_info['education'].unique() ,credit_info['family_status'].unique(),credit_info['gender'].unique(),
credit_info['income_type'].unique(),credit_info['purpose'].unique()]
print(inuque_values)

[array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75]), array([ 1,  0,  3,  2, -1,  4, 20,  5]), array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object), array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object), array(['F', 'M', 'XNA'], dtype=object), array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object), array(['purchase of the house', '

In [47]:
print((credit_info['dob_years'] == 0).sum())
print((credit_info['children'] == -1).sum())
print((credit_info['children'] == 20).sum())

101
47
76


### Conclusion

After analazing the table we can see following issues to encounter:
1. two columns have equal amount (~10%) of the missing values: days_employed  and total_income;
2. values for days of the employment are not integer, there are negative and unrealistically big values;
3. education column contains duplicates;
4. missing values (<1%) in age column: age can't be 0;
5. odd data (<1%) in 'children' column: -1 and 20 
6. 1 missing value in 'gender' column

## Data preprocessing

### Processing missing values

We are replacing missing data in total income column using the median method. It is more suitable since we have some extreme values in the data. For more accuracy we use median for groups with the same education and employment type.
Since the amount of missing data in days employed and total income is the same, we can assume that it might've been the result of some technical issue.

In [48]:
#filling in total income missing values based on average income of people with the same income type and education:
credit_info['total_income']=credit_info['total_income'].fillna(credit_info.groupby(['education','income_type'])['total_income'].transform('median'))
print(credit_info.head(70))
print(credit_info['total_income'].isnull().sum())

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
..       ...            ...        ...                  ...           ...   
65         0            NaN         21  secondary education             1   
66         0    -916.428829         28  secondary education             1   
67         0            NaN         52    bachelor's degree             0   
68         2   -2152.475526         46    bachelor's degree             0   
69         0    -779.228449         23  secondary education             1   

        family_status  family_status_id gender income_type  debt  \
0      

The amount of days people are employed can influence their income, 
but in our case it won't help us fill the missing values within the data.

The amount of days people are employed can also give us an information about  a customer's reliability,  however this parameter is not an object of interest in our research.

We can drop this data for now, but might go back to it later.

In [49]:
#drop days_employed column
credit_info=credit_info.drop(['days_employed'], axis=1)
print(credit_info.head(10))

   children  dob_years            education  education_id      family_status  \
0         1         42    bachelor's degree             0            married   
1         1         36  secondary education             1            married   
2         0         33  Secondary Education             1            married   
3         3         32  secondary education             1            married   
4         0         53  secondary education             1  civil partnership   
5         0         27    bachelor's degree             0  civil partnership   
6         0         43    bachelor's degree             0            married   
7         0         50  SECONDARY EDUCATION             1            married   
8         2         35    BACHELOR'S DEGREE             0  civil partnership   
9         0         41  secondary education             1            married   

   family_status_id gender income_type  debt  total_income  \
0                 0      F    employee     0     40620.10

We will use the same method for filling missing values in age column as for the total income column.

In [50]:
#filling 0 in years based on education and income type
credit_info['dob_years']=credit_info['dob_years'].replace(0, np.nan)
print(credit_info['dob_years'].isnull().sum())
credit_info['dob_years']=credit_info['dob_years'].fillna(credit_info.groupby(['education','income_type'])['dob_years'].transform('median'))
print(credit_info['dob_years'].isnull().sum())
print((credit_info['dob_years'] == 0).sum())

101
0
0


The percentage of data that would be considered missing in children column is less than 1 %, therefore we can make some assumptions without damaging the analyses. We will turn negative values into positive ones.
Values that equal  20  will be considered as a result of a mistype and replaced with 2. 
(Numbers 2 and 0 are close on a keyboard).

In [51]:
#getting rid of unrealistic data in children column
credit_info['children'] = abs(credit_info['children'])
credit_info['children'] =credit_info['children'].replace(20, 2)
credit_info['children'].unique()


array([1, 0, 3, 2, 4, 5])

Only one missing value in gender column: we will replace it with random gender. It shouldn't affect the research.

In [52]:
print(credit_info['gender'].value_counts())
credit_info['gender'] =credit_info['gender'].replace('XNA', 'F')

print(credit_info['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
F    14237
M     7288
Name: gender, dtype: int64


### Conclusion

All the missing values were replaced or droped.


### Data type replacement

In [53]:
credit_info['total_income']=credit_info['total_income'].astype(int)
credit_info['dob_years']=credit_info['dob_years'].astype(int)
credit_info['education']=credit_info['education'].astype(str)
credit_info['purpose']=credit_info['purpose'].astype(str)
print(credit_info.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB
None


### Conclusion

All the types of the numeric values were changed to integer type using astype() method.

### Processing duplicates

In [54]:
#education
credit_info['education']=credit_info['education'].str.lower()
print(credit_info['education'].value_counts())
print(credit_info['education'].value_counts().sum())


secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64
21525


In [55]:
print(credit_info.duplicated().sum())
print(credit_info.loc[credit_info.duplicated()])

54
       children  dob_years            education  education_id  \
2849          0         41  secondary education             1   
4182          1         34    bachelor's degree             0   
4851          0         60  secondary education             1   
5557          0         58  secondary education             1   
7808          0         57  secondary education             1   
8583          0         58    bachelor's degree             0   
9238          2         34  secondary education             1   
9528          0         66  secondary education             1   
9627          0         56  secondary education             1   
10462         0         62  secondary education             1   
10697         0         40  secondary education             1   
10864         0         62  secondary education             1   
10994         0         62  secondary education             1   
11791         0         47  secondary education             1   
12373         0       

In [56]:
credit_info = credit_info.drop_duplicates().reset_index(drop = True)
print(credit_info.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 11 columns):
children            21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB
None


### Conclusion

1. Duplicates in education column were caused by using different cases during typing. Issue is solved by lower() function
2. Duplicates within the rows might be an issue of some technical kind or real data from people with the same background/life situation. 54 rows - is less than 1% of data and it won't take too much memory to leave it as it is.

### Categorizing Data

# Creating categories for the purpose data:

With we look closer into the purposes we can define four big categories: wedding, education, car, some real estate transactions. 
Within the last category we can choose some subcategories like commercial and personal purposes but it's not always clear for us from the description. 
Therefore we will stick to the bigger four categories.

In [57]:
print(credit_info['purpose'].value_counts())


wedding ceremony                            793
having a wedding                            773
to have a wedding                           769
real estate transactions                    675
buy commercial real estate                  662
housing transactions                        652
buying property for renting out             652
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      625
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [58]:
from nltk.stem import SnowballStemmer

english_stemmer = SnowballStemmer('english')

from nltk.stem import WordNetLemmatizer

from collections import Counter
lemmatizer = WordNetLemmatizer()
 
print(lemmatizer.lemmatize("education"))

education


In [59]:
from nltk.stem import SnowballStemmer

english_stemmer = SnowballStemmer('english')

from nltk.stem import WordNetLemmatizer

from collections import Counter
lemmatizer = WordNetLemmatizer()

purpose_type = []
for value in credit_info["purpose"]:
    
    word_list = nltk.word_tokenize(value)
    lemmas = [lemmatizer.lemmatize(w) for w in word_list]
    
    if 'wedding' in lemmas:
        
        purpose_type.append("wedding")
        
    elif 'car' in lemmas:
        purpose_type.append("car")
        
    elif 'educated' in lemmas or 'education'in lemmas or 'university' in lemmas:
        purpose_type.append("education")
        
    else:
        purpose_type.append("housing")
       
    
    
credit_info["purpose_type"] = purpose_type   
print(credit_info.head(30))
    


    children  dob_years            education  education_id      family_status  \
0          1         42    bachelor's degree             0            married   
1          1         36  secondary education             1            married   
2          0         33  secondary education             1            married   
3          3         32  secondary education             1            married   
4          0         53  secondary education             1  civil partnership   
5          0         27    bachelor's degree             0  civil partnership   
6          0         43    bachelor's degree             0            married   
7          0         50  secondary education             1            married   
8          2         35    bachelor's degree             0  civil partnership   
9          0         41  secondary education             1            married   
10         2         36    bachelor's degree             0            married   
11         0         40  sec

In [60]:
print(credit_info['purpose_type'].value_counts())

housing      10814
car           4308
education     4014
wedding       2335
Name: purpose_type, dtype: int64


In [61]:
def purpose_id(purpose):
       
    if purpose == 'housing':
        return 0    
    if purpose =='car':
        return 1    
    if purpose =='education':
        return 2
    
    return 3

credit_info['purpose_id'] = credit_info['purpose_type'].apply(purpose_id)

print(credit_info['purpose_id'].value_counts())

0    10814
1     4308
2     4014
3     2335
Name: purpose_id, dtype: int64


# Creating categories for income:


In [62]:
print(credit_info['total_income'].describe())

count     21471.000000
mean      26471.348703
std       15723.365182
min        3306.000000
25%       17202.500000
50%       22912.000000
75%       31720.000000
max      362496.000000
Name: total_income, dtype: float64


In [63]:
print(pd.cut(credit_info['total_income'], 40).value_counts())

(21265.5, 30245.25]      6831
(12285.75, 21265.5]      6656
(30245.25, 39225.0]      3047
(2946.81, 12285.75]      1981
(39225.0, 48204.75]      1493
(48204.75, 57184.5]       658
(57184.5, 66164.25]       343
(66164.25, 75144.0]       175
(75144.0, 84123.75]       109
(84123.75, 93103.5]        55
(102083.25, 111063.0]      31
(93103.5, 102083.25]       29
(111063.0, 120042.75]      15
(129022.5, 138002.25]      10
(173921.25, 182901.0]       6
(120042.75, 129022.5]       5
(138002.25, 146982.0]       4
(155961.75, 164941.5]       4
(272698.5, 281678.25]       3
(146982.0, 155961.75]       3
(200860.5, 209840.25]       2
(191880.75, 200860.5]       2
(164941.5, 173921.25]       2
(254739.0, 263718.75]       1
(245759.25, 254739.0]       1
(209840.25, 218820.0]       1
(353516.25, 362496.0]       1
(227799.75, 236779.5]       1
(344536.5, 353516.25]       1
(182901.0, 191880.75]       1
(308617.5, 317597.25]       0
(335556.75, 344536.5]       0
(326577.0, 335556.75]       0
(317597.25

As we can see income data has some extreme values, so cutting it equally in 4 categories wouldn't be a good option. Instead we can define the range of the average income level (income that has most of people within our data). Based on average income level we can also define below average, and more than average income levels:
1. less than 15000
2. 15000 - 25000
3. 25000 - 60000
4. more than 60000

In [64]:
def income_category(income):
       
    if income <= 15000:
        return 'low_income'
    
    if income <= 25000:
        return 'lower_average_income'
    
    if income <= 60000:
        return 'higher_average_income'
    
   # if income <= 100000:
      #  return 'higher_income'
    
    return 'upper-class_income' 

credit_info['income_type'] = credit_info['total_income'].apply(income_category)
print(credit_info.head(10))
print(credit_info['income_type'].value_counts())

   children  dob_years            education  education_id      family_status  \
0         1         42    bachelor's degree             0            married   
1         1         36  secondary education             1            married   
2         0         33  secondary education             1            married   
3         3         32  secondary education             1            married   
4         0         53  secondary education             1  civil partnership   
5         0         27    bachelor's degree             0  civil partnership   
6         0         43    bachelor's degree             0            married   
7         0         50  secondary education             1            married   
8         2         35    bachelor's degree             0  civil partnership   
9         0         41  secondary education             1            married   

   family_status_id gender            income_type  debt  total_income  \
0                 0      F  higher_average_inc

In [65]:
def income_id(income):
       
    if income == 'low_income':
        return 0    
    if income =='lower_average_income':
        return 1    
    if income =='higher_average_income':
        return 2 
   # if income =='higher_income':
       # return 3
    
    return 3

credit_info['income_id'] = credit_info['income_type'].apply(income_id)
print(credit_info.head(10))
print(credit_info['income_id'].value_counts())

   children  dob_years            education  education_id      family_status  \
0         1         42    bachelor's degree             0            married   
1         1         36  secondary education             1            married   
2         0         33  secondary education             1            married   
3         3         32  secondary education             1            married   
4         0         53  secondary education             1  civil partnership   
5         0         27    bachelor's degree             0  civil partnership   
6         0         43    bachelor's degree             0            married   
7         0         50  secondary education             1            married   
8         2         35    bachelor's degree             0  civil partnership   
9         0         41  secondary education             1            married   

   family_status_id gender            income_type  debt  total_income  \
0                 0      F  higher_average_inc

# Children

In [66]:
print(credit_info['children'].value_counts())

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


In [67]:
def children_count(children):
       
    if children == 0:
        return 'No children'    
    if children ==1:
        return 'One child' 
    
    if children ==2:
        return 'Two children'
    
    if children ==3:
        return 'Three children'
    
    if children ==4:
        return 'Four children'
    
    return 'Five Children'

credit_info['children_status'] = credit_info['children'].apply(children_count)

print(credit_info['children_status'].value_counts())

No children       14107
One child          4856
Two children       2128
Three children      330
Four children        41
Five Children         9
Name: children_status, dtype: int64


### Conclusion

The categories that are needed within our research are the number of children, marital status, income level, and purpose.
The categories for the children and the marital status were used as they are.
The income was split into four categories with the main category being an average income.
The purpose was split into four types: wedding, car purchase, education, and housing. 

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [68]:
kids = pd.pivot_table(credit_info,index = 'children', columns='debt', 
                               values='children_status',aggfunc='count',margins=True).reset_index()

kids['default_rate']=(kids[1]/kids['All'])*100
kids[1] = kids[1].fillna(0)
kids['default_rate'] = kids['default_rate'].fillna('No default debt')
print(kids)

debt children        0       1    All     default_rate
0           0  13044.0  1063.0  14107          7.53527
1           1   4411.0   445.0   4856          9.16392
2           2   1926.0   202.0   2128          9.49248
3           3    303.0    27.0    330          8.18182
4           4     37.0     4.0     41           9.7561
5           5      9.0     0.0      9  No default debt
6         All  19730.0  1741.0  21471          8.10861


### Conclusion

Based on the data we were provided with we can conclude that people without kids are more reliable. 
People with one or two children have approximately the same reliability level, and it is lower than the one of people without kids.
The number of people with more than 2 kids is not enough to make an overall conclusion on how the number of kids affects the reliability.

- Is there a relation between marital status and repaying a loan on time?

In [69]:
marital_status = pd.pivot_table(credit_info,index = 'family_status', columns='debt', 
                               values='family_status_id',aggfunc='count',margins=True).reset_index()
print(marital_status)

debt      family_status      0     1    All
0     civil partnership   3775   388   4163
1              divorced   1110    85   1195
2               married  11413   931  12344
3             unmarried   2536   274   2810
4       widow / widower    896    63    959
5                   All  19730  1741  21471


In [70]:
marital_status['default_rate']=(marital_status[1]/marital_status['All'])*100
print(marital_status)

debt      family_status      0     1    All  default_rate
0     civil partnership   3775   388   4163      9.320202
1              divorced   1110    85   1195      7.112971
2               married  11413   931  12344      7.542126
3             unmarried   2536   274   2810      9.750890
4       widow / widower    896    63    959      6.569343
5                   All  19730  1741  21471      8.108612


### Conclusion

According to the provided data widows/widowers are the most reliable people based on the marital status.
They are followed by divorced and married people.
The most unreliable ones are in a civil partnership or unmarried.

- Is there a relation between income level and repaying a loan on time?

In [71]:
income_level = pd.pivot_table(credit_info,index = 'income_type', columns='debt', 
                               values='income_id',aggfunc='count',margins=True).reset_index()

income_level['default_rate']=(income_level[1]/income_level['All'])*100

print(income_level)

debt            income_type      0     1    All  default_rate
0     higher_average_income   8035   681   8716      7.813217
1                low_income   3448   298   3746      7.955152
2      lower_average_income   7611   724   8335      8.686263
3        upper-class_income    636    38    674      5.637982
4                       All  19730  1741  21471      8.108612


### Conclusion

According to the provided data people with upper-class income are more reliable. (income more than 600000)
However, much fewer people from this category are taking the loan compared to others. 
The most unreliable people are the ones thaT earn slightly more than low income: income 15000-25000
People with higher average income (25000-60000) and low income have approximately the same level of reliability, though people with lower income are rarer to take a loan.

- How do different loan purposes affect on-time repayment of the loan?

In [72]:
purpose = pd.pivot_table(credit_info,index = 'purpose_type', columns='debt', 
                               values='purpose_id',aggfunc='count',margins=True).reset_index()

purpose['default_rate']=(purpose[1]/purpose['All'])*100

print(purpose)

debt purpose_type      0     1    All  default_rate
0             car   3905   403   4308      9.354689
1       education   3644   370   4014      9.217738
2         housing  10032   782  10814      7.231367
3         wedding   2149   186   2335      7.965739
4             All  19730  1741  21471      8.108612


### Conclusion

The most reliable people are the ones that take the loan for housing purposes. although as it was mentioned housing in our case is a very large category, with more information from the customer we could've decided it in smaller groups to get more accurate results.
Another reliable group consists of the people who take the loan for their wedding.
The most unreliable groups are the ones that have car or education as purposes.

## General conclusion

The data was preprocessed: missing values were filled where it was possible, days employed were dropped out since they do not have an influence on our research. We got rid of the duplicates. Reassigned the data type for real values and made them integer.
Data were categorized according to the research purposes.
The reliability of the categories was analyzed according to each parameter of interest.
It is recommended to gather more data within certain groups to improve the accuracy of our conclusions.